In [1]:
# files
import utils

# packages
import numpy as np
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
import time

Using TensorFlow backend.


In [2]:
MODEL_CLASSES = {
    'bert': (           BertForSequenceClassification,          BertTokenizer,          'bert-base-uncased'),
}

model_class, tokenizer, pretrained_model = MODEL_CLASSES['bert']
tokenizer = tokenizer.from_pretrained(pretrained_model)

In [3]:
def create_sentences(csv_file):
    '''
    Dataset creation containing the questions with their tokenization corresponding to the model used.
    - input: CSV filename (raw data)
    - output: dataframe
    '''
    data = pd.read_csv(csv_file).fillna("")
    tr_qids = pd.Series(data['qid1'].tolist() + data['qid2'].tolist())
    unique_dic = tr_qids.value_counts()
    data = np.array(data)
    
    sentences = [ [] for _ in range(len(unique_dic)) ]
    for i in range(len(data)):
        for j in range(1,3):
            ind = data[i,j]
            if ind == 174363+1 or ind == 493339+1:
                print(ind, data[i,j+2])
            if not sentences[ind-1]:
                question = data[i,j+2]
                embedding = tokenizer.encode(question, add_special_tokens=False) 
                sentences[ind-1] = [question, embedding]
            if ind == 174363+1 or indS == 493339+1:
                print(ind, sentences[ind-1])
    sentences = pd.DataFrame(sentences)
    return sentences

In [4]:
file_IN = "data/train.csv"
file_OUT = "data/sentences.csv"

df = create_sentences(file_IN)
#utils.WriteCSV(df, file_OUT)

174364 
174364 ['', []]
174364 
174364 ['', []]
493340 
493340 ['', []]


In [55]:
print(df.iloc[174363,:])
print(df.loc[493339,:])

question       
embedding    []
Name: 174363, dtype: object
question       
embedding    []
Name: 493339, dtype: object
